In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "a5fc98ca",
   "metadata": {
    "vscode": {
     "languageId": "plaintext"
    }
   },
   "outputs": [],
   "source": [
    "from dotenv import load_dotenv\n",
    "import os \n",
    "from langchain_community.llms import Ollama\n",
    "from langchain_openai.chat_models import ChatOpenAI\n",
    "from langchain.prompts import PromptTemplate\n",
    "from langchain_core.output_parsers import StrOutputParser\n",
    "from langchain.text_splitter import RecursiveCharacterTextSplitter\n",
    "from langchain_community.document_loaders import TextLoader\n",
    "from langchain_openai.embeddings import OpenAIEmbeddings\n",
    "from langchain_community.vectorstores import FAISS\n",
    "from langchain_core.runnables import RunnablePassthrough, RunnableParallel\n",
    "\n",
    "\n",
    "ollama_llm = Ollama(model = 'llama3')\n",
    "\n",
    "load_dotenv()\n",
    "API_KEY = os.getenv('OPENAI_API_KEY')\n",
    "Model = 'gpt-3.5-turbo'\n",
    "gpt_llm = ChatOpenAI(api_key = API_KEY,model= Model)\n",
    "\n",
    "gpt_llm.invoke('what is a bot')\n",
    "\n",
    "parser = StrOutputParser()\n",
    "gpt_chain = gpt_llm|parser\n",
    "gpt_chain.invoke('what is a bot')\n",
    "\n",
    "loader = TextLoader('data.txt',encoding = 'utf-8')\n",
    "document = loader.load()\n",
    "\n",
    "spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)\n",
    "chunks = spliter.split_documents(document)\n",
    "chunks[3]\n",
    "\n",
    "vector_storage = FAISS.from_documents(chunks,OpenAIEmbeddings())\n",
    "retriever = vector_storage.as_retriever()\n",
    "\n",
    "retriever.invoke('what is the pricing for Bhuman')\n",
    "\n",
    "template = (\"\"\"\n",
    "You are AI-powered chatbot designed to provide \n",
    "information and assistance for customers\n",
    "based on the context provided to you only. \n",
    "            \n",
    "Context:{context}\n",
    "Question:{question}\n",
    "\"\"\")\n",
    "\n",
    "prompt = PromptTemplate.from_template(template=template)\n",
    "prompt.format(\n",
    "    context = ' Here is a context to use',\n",
    "    question = ' This is a question to answer'\n",
    ")\n",
    "\n",
    "result = RunnableParallel(context = retriever,question = RunnablePassthrough())\n",
    "chain = result |prompt | gpt_llm | parser\n",
    "\n",
    "chain.invoke('What is the pricing plan for Bhuman')\n",
    "\n",
    "chain.invoke('What is Bhuman')\n",
    "\n",
    "chain.invoke('My video is not clear')\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}